In [4]:
import os
import re
import csv

# Define the CSV file containing the file identifiers

csv_file = os.path.join(r'C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2', 'hadm_id_for_discharge_me_scoring.csv')

# Read the file identifiers from the CSV file
identifiers = []
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the first row (column name)
    for row in reader:
        identifiers.extend([identifier + '_inputs' for identifier in row])



In [27]:

# Define the headers and candidate titles
headers = ['BHC', 'Translational_Issues', 'Medications', 'Discharge_Instructions']
candidate_titles = {
    'BHC': ['Brief Hospital Course:', 'BHC:', 'Course:'],
    'Translational_Issues': ['Translational Issue' , 'Translational Issues:'],
    'Medications': ['Medication List', 'Medications on Admission', 'Admission Medications'],
    'Discharge_Instructions': ['Discharge Instructions', 'Discharge Instructions:', 'Discharge Instruction'],
    'Disposition': ['Disposition:', 'Dispo:'],
    'Discharge_dx': ['Discharge Diagnosis:', 'Diagnosis on Discharge:', 'discharge dx']    
}


# patterns =[ 
#     rf'({"|".join(candidate_titles["BHC"])})(.*?)(?={"|".join(candidate_titles["Disposition"])})',
#     rf'({"|".join(candidate_titles["BHC"])})(.*?)(?={"|".join(candidate_titles["Discharge_dx"])})',
#     rf'({"|".join(candidate_titles["BHC"])})(.*?)(?={"|".join(candidate_titles["Medications"])})',
#     rf'({"|".join(candidate_titles["BHC"])})(.*?)(?={"|".join(candidate_titles["Discharge_Instructions"])})'
# ]

# patterns = { 'BHC': r'(BHC:|Brief Hospital Course:)(.*?)(?=Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Discharge Diagnosis:$)',

headings_regex = {
  'BHC': r'(BHC:|Brief Hospital Course:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|Disposition:|Dispo:|Discharge Instructions:|Patient Instructions:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',
  'Discharge Instructions': r'(Discharge Instructions:|Patient Instructions:)(.*?)(?=CC:|Chief Complaint:|Reason for admission:|Reason for hospital admission:|Service:|Specialty:|Unit:|Major Surgical or Invasive Procedure:|Major Medical Procedures:|Major Procedures:|Major Surgeries:|HPI:|History of Present Illness:|PMH:|Past Medical History:|Social History:|SOC:|SH:|Family History:|PFH:|FH:|Past Surgical History:|PSH:|Problem List:|Problems:|Physical Exam:|PE:|Medication lists:|Admission Medications:|Medications on Admission:|Discharge Medications:|Medications on Discharge:|Pertinent Results:|Pertinent imaging:|Pertinent Labs:|Pertinent Microbiology:|Pertinent Micro:|Results:|BHC:|Brief Hospital Course:|Disposition:|Dispo:|Followup Instructions:|Transitional Issues:|Pertinent Results:|Pertinent Findings:|Discharge Diagnosis:$)',    
    }

print(patterns)


['(Brief Hospital Course:|BHC:|Course:)(.*?)(?=Disposition:|Dispo:)', '(Brief Hospital Course:|BHC:|Course:)(.*?)(?=Discharge Diagnosis:|Diagnosis on Discharge:|discharge dx)', '(Brief Hospital Course:|BHC:|Course:)(.*?)(?=Medication List|Medications on Admission|Admission Medications)', '(Brief Hospital Course:|BHC:|Course:)(.*?)(?=Discharge Instructions|Discharge Instructions:|Discharge Instruction)']


In [30]:

# Define the masking function
def mask_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    # Perform the masking function (same as before)
        for heading, regex in headings_regex.items():
            match = re.search(regex, text, re.DOTALL |re.IGNORECASE)
            # match = re.search(pattern, text, re.DOTALL | re.I )
            if match:
                text_to_mask = match.group(2).strip()
                if text_to_mask is not None:
                    text = text.replace(text_to_mask, '***Masked***')
 
    # Write the masked text to a new file in the "masked_text_inputs" subdirectory
    masked_dir = 'masked_text_inputs'
    os.makedirs(masked_dir, exist_ok=True)  # Create the subdirectory if it doesn't exist
    masked_file_path = os.path.join(masked_dir, os.path.basename(file_path) + '_masked.txt')
    with open(masked_file_path, 'w') as file:
        file.write(text)

# Search for files with matching identifiers
data_dir = os.path.join(r'C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2', 'all_text_data_by_hadm_id')

for root, dirs, files in os.walk(data_dir):
    for file in files:
        for identifier in identifiers:
            if identifier in file:
                file_path = os.path.join(root, file)
                mask_text(file_path)
                print(f"Masked file: {file_path}")

Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_20063495_inputs.txt
Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_20070958_inputs.txt
Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_20154746_inputs.txt
Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_20172512_inputs.txt
Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_20202032_inputs.txt
Masked file: C:\Users\s.wendelken\OneDrive - Northeastern University\Projects\AI EHR\Discharge_me_data\test_phase_2\all_text_data_by_hadm_id\hadm_id_202266